<a href="https://colab.research.google.com/github/kajalpanda1/ML-Assignments/blob/master/Voting_Ensemble_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#importing pima indian diabetes dataset
df = pd.read_csv('https://raw.githubusercontent.com/kajalpanda1/DatasetRepository/master/pima-indians-diabetes.csv',header=None)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
#split feature and target sets
x=df.iloc[:,0:-1]
print(x.shape)

y=df.iloc[:,-1]
print(y.shape)

(768, 8)
(768,)


In [7]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x=ss.fit_transform(x)

In [9]:
#split data into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

In [10]:
#BUILDING THE MODELS
#K-NN Classifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

#create new a knn model
knn = KNeighborsClassifier()

#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}

#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)

#fit model to training data
knn_gs.fit(X_train, y_train)

#save best model
knn_best = knn_gs.best_estimator_

#check best n_neigbors value
print(knn_gs.best_params_)


{'n_neighbors': 17}


In [11]:
#RAMDOM FOREST
from sklearn.ensemble import RandomForestClassifier

#create a new random forest classifier
rf=RandomForestClassifier()

#create a dictionary of all values we want to test for n_estimators
params_rf={'n_estimators': [50,100,200],'max_leaf_nodes':[7,8,9,10],'max_features':[4,5,6,7]}

#use gridsearch to test all values for n_estimators
rf_gs=GridSearchCV(rf, params_rf, cv=5)

#fit model to training data
rf_gs.fit(X_train, y_train)

#save best model
rf_best=rf_gs.best_estimator_

#check best n_estimators value
print(rf_gs.best_params_)

{'max_features': 4, 'max_leaf_nodes': 8, 'n_estimators': 50}


In [12]:
#LogisticRegression

from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg=LogisticRegression(solver='liblinear')

#fit the model to the training data
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_train, y_train)))
print('rf: {}'.format(rf_best.score(X_train, y_train)))
print('log_reg: {}'.format(log_reg.score(X_train, y_train)))

knn: 0.6775244299674267
rf: 0.6824104234527687
log_reg: 0.6530944625407166


In [14]:
#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))

knn: 0.6233766233766234
rf: 0.6363636363636364
log_reg: 0.6493506493506493


In [16]:
#VOTING CLASSIFIER
from sklearn.ensemble import VotingClassifier

#create a dictionary of our models
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', log_reg)]

#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

#fit model to training data
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=17,
                                                   p=2, weights='uniform')),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=4,
                                                     max_leaf_nodes=8,
     

In [17]:
#Training and testing accuracy of voting ensemble
print('Training:',ensemble.score(X_train,y_train))
print('Testing:',ensemble.score(X_test, y_test))

Training: 0.6547231270358306
Testing: 0.6493506493506493
